In [4]:
from transformers import AutoTokenizer
from torch.utils.data import Dataset
import torch
import pandas as pd
from tqdm import tqdm

In [5]:
train_df = pd.read_csv('../input/feedback-prize-2021/train.csv')

In [6]:
tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096', add_prefix_space=True)


In [8]:
test_names, train_texts = [], []
for f in tqdm(list(os.listdir('../input/feedback-prize-2021/train'))):
    test_names.append(f.replace('.txt', ''))
    train_texts.append(open('../input/feedback-prize-2021/train/' + f, 'r', encoding='utf-8').read())
train_text_df = pd.DataFrame({'id': test_names, 'text': train_texts})
train_text_df.head()

100%|██████████| 15594/15594 [00:00<00:00, 23580.67it/s]


,id,text
0,4BCA0580352F,Isn't it wonderful to know that you get a chan...
1,29A8E6DA6F04,"One uses a car to go to the store, pick someon..."
2,17E8BB6C550B,Driverless cars do seem like the thing today's...
3,319B2511943D,Have you ever thought your choice wasn't the b...
4,C9B1827498E4,"Imagine you're sitting at home, watching a sho..."


In [9]:
all_entities = []
for ii,i in enumerate(train_text_df.iterrows()):
    if ii%100==0: print(ii,', ',end='')
    total = i[1]['text'].split().__len__()
    entities = ["O"]*total
    for j in train_df[train_df['id'] == i[1]['id']].iterrows():
        discourse = j[1]['discourse_type']
        list_ix = [int(x) for x in j[1]['predictionstring'].split(' ')]
        entities[list_ix[0]] = f"B-{discourse}"
        for k in list_ix[1:]: entities[k] = f"I-{discourse}"
    all_entities.append(entities)
train_text_df['entities'] = all_entities

0 , 100 , 200 , 300 , 400 , 500 , 600 , 700 , 800 , 900 , 1000 , 1100 , 1200 , 1300 , 1400 , 1500 , 1600 , 1700 , 1800 , 1900 , 2000 , 2100 , 2200 , 2300 , 2400 , 2500 , 2600 , 2700 , 2800 , 2900 , 3000 , 3100 , 3200 , 3300 , 3400 , 3500 , 3600 , 3700 , 3800 , 3900 , 4000 , 4100 , 4200 , 4300 , 4400 , 4500 , 4600 , 4700 , 4800 , 4900 , 5000 , 5100 , 5200 , 5300 , 5400 , 5500 , 5600 , 5700 , 5800 , 5900 , 6000 , 6100 , 6200 , 6300 , 6400 , 6500 , 6600 , 6700 , 6800 , 6900 , 7000 , 7100 , 7200 , 7300 , 7400 , 7500 , 7600 , 7700 , 7800 , 7900 , 8000 , 8100 , 8200 , 8300 , 8400 , 8500 , 8600 , 8700 , 8800 , 8900 , 9000 , 9100 , 9200 , 9300 , 9400 , 9500 , 9600 , 9700 , 9800 , 9900 , 10000 , 10100 , 10200 , 10300 , 10400 , 10500 , 10600 , 10700 , 10800 , 10900 , 11000 , 11100 , 11200 , 11300 , 11400 , 11500 , 11600 , 11700 , 11800 , 11900 , 12000 , 12100 , 12200 , 12300 , 12400 , 12500 , 12600 , 12700 , 12800 , 12900 , 13000 , 13100 , 13200 , 13300 , 13400 , 13500 , 13600 , 13700 , 13800 , 

In [10]:
train_text_df

,id,text,entities
0,4BCA0580352F,Isn't it wonderful to know that you get a chan...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
1,29A8E6DA6F04,"One uses a car to go to the store, pick someon...","[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
2,17E8BB6C550B,Driverless cars do seem like the thing today's...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
3,319B2511943D,Have you ever thought your choice wasn't the b...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
4,C9B1827498E4,"Imagine you're sitting at home, watching a sho...","[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
...,...,...,...
15589,F2DFE12EA3B4,Many students have expressed that projects sho...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
15590,5976F16FE9D2,In the 1960's driverless cars seemed more as l...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
15591,D4CF0DF858F1,"Looking back on that last tough decision, do y...","[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
15592,DA8115F12B50,Everyday a great many people drive their cars ...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."


In [11]:
output_labels = ['O', 'B-Lead', 'I-Lead', 'B-Position', 'I-Position', 'B-Claim', 'I-Claim', 'B-Counterclaim', 'I-Counterclaim', 
          'B-Rebuttal', 'I-Rebuttal', 'B-Evidence', 'I-Evidence', 'B-Concluding Statement', 'I-Concluding Statement']

labels_to_ids = {v:k for k,v in enumerate(output_labels)}
ids_to_labels = {k:v for k,v in enumerate(output_labels)}
disc_type_to_ids = {'Lead':(1,2),'Position':(3,4),'Claim':(5,6),'Counterclaim':(7,8),'Rebuttal':(9,10),'Evidence':(11,12),'Concluding Statement':(13,14)}

In [12]:
import numpy as np
def split_mapping(unsplit):
    splt = unsplit.split()
    offset_to_wordidx = np.full(len(unsplit),-1)
    txt_ptr = 0
    for split_index, full_word in enumerate(splt):
        while unsplit[txt_ptr:txt_ptr + len(full_word)] != full_word:
            txt_ptr += 1
        offset_to_wordidx[txt_ptr:txt_ptr + len(full_word)] = split_index
        txt_ptr += len(full_word)
    return offset_to_wordidx



In [47]:
from scipy import stats
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len, get_wids):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.get_wids = get_wids # for validation

  def __getitem__(self, index):
        # GET TEXT AND WORD LABELS 
        text = self.data.text[index]        
        word_labels = self.data.entities[index] if not self.get_wids else None

        # TOKENIZE TEXT
        encoding = self.tokenizer(text,
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        
        word_ids = encoding.word_ids()  
        split_word_ids = np.full(len(word_ids),-1)
        offset_to_wordidx = split_mapping(text)
        offsets = encoding['offset_mapping']
        
        # CREATE TARGETS AND MAPPING OF TOKENS TO SPLIT() WORDS
        label_ids = []
        # Iterate in reverse to label whitespace tokens until a Begin token is encountered
        for token_idx, word_idx in reversed(list(enumerate(word_ids))):
            
            if word_idx is None:
                if not self.get_wids: label_ids.append(-100)
            else:
                if offsets[token_idx] != (0,0):
                    #Choose the split word that shares the most characters with the token if any
                    split_idxs = offset_to_wordidx[offsets[token_idx][0]:offsets[token_idx][1]]
                    split_index = stats.mode(split_idxs[split_idxs != -1]).mode[0] if len(np.unique(split_idxs)) > 1 else split_idxs[0]
                    
                    if split_index != -1: 
                        if not self.get_wids: label_ids.append( labels_to_ids[word_labels[split_index]] )
                        split_word_ids[token_idx] = split_index
                    else:
                        # Even if we don't find a word, continue labeling 'I' tokens until a 'B' token is found
                        if label_ids and label_ids[-1] != -100 and ids_to_labels[label_ids[-1]][0] == 'I':
                            split_word_ids[token_idx] = split_word_ids[token_idx + 1]
                            if not self.get_wids: label_ids.append(label_ids[-1])
                        else:
                            if not self.get_wids: label_ids.append(-100)
                else:
                    if not self.get_wids: label_ids.append(-100)
        
        encoding['labels'] = list(reversed(label_ids))

        # CONVERT TO TORCH TENSORS
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        if self.get_wids: 
            item['wids'] = torch.as_tensor(split_word_ids)
        return item

  def __len__(self):
        return self.len

In [48]:
training_set = dataset(train_text_df, tokenizer, 1024, False)

In [49]:
from torch.utils.data import DataLoader
training_loader = DataLoader(training_set, batch_size = 1, shuffle=False, num_workers=2, pin_memory=True)